# model test

## Tensorized

In [1]:
def my_func():
    a = 1
    b = 2
    c = 3
    return (a,b),c

x = my_func()
print(x)
y1, _ = my_func()
print(y1)
z1, z2, z3 = my_func()
print(z1,z2,z3)
    

((1, 2), 3)
(1, 2)


ValueError: not enough values to unpack (expected 3, got 2)

In [8]:
import ast

temp = '{0 : 1,100 : 2,200 : 3}'
temp_list = ast.literal_eval(temp)

print(type(temp_list))
print(temp_list)

<class 'dict'>
{0: 1, 100: 2, 200: 3}


In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

optimizer_name = 'ZO_SCD_esti'
h_smooth = 0.008

grads_path=os.path.join('./figs/' + optimizer_name + '/h_'+str(h_smooth)+'.pth')

grads = torch.load(grads_path)


X=np.array([[0,3,2,4],[5,4,7,8],[9,16,8,5],[13,3,4,16],[6,18,1,20]])
A = np.arange(0, 100).reshape(10, 10)
ax = plt.matshow(X)
plt.colorbar(ax.colorbar, fraction=0.025)
plt.title("matrix X");
plt.show()